In [15]:
import numpy as np
import pandas as pd
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB
# to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [48]:
import matplotlib.pyplot as plt
from sklearn.linear_model import RidgeCV, LassoCV

In [17]:
train = pd.read_csv('HW1/train.csv')

In [18]:
test = pd.read_csv('HW1/test.csv')

In [19]:
sample_submission = pd.read_csv ("HW1/sample_submission.csv")

In [20]:
pd.options.display.max_columns = 200

Альтернативное решение, с использованием максимального количества факторов. Сливаем две выборки воедино. 

In [21]:
testID = test['Id']

data = pd.concat([train.drop('SalePrice', axis=1), test], keys=['train', 'test'])
data.pop('Id')

train  0          1
       1          2
       2          3
       3          4
       4          5
       5          6
       6          7
       7          8
       8          9
       9         10
       10        11
       11        12
       12        13
       13        14
       14        15
       15        16
       16        17
       17        18
       18        19
       19        20
       20        21
       21        22
       22        23
       23        24
       24        25
       25        26
       26        27
       27        28
       28        29
       29        30
               ... 
test   1429    2890
       1430    2891
       1431    2892
       1432    2893
       1433    2894
       1434    2895
       1435    2896
       1436    2897
       1437    2898
       1438    2899
       1439    2900
       1440    2901
       1441    2902
       1442    2903
       1443    2904
       1444    2905
       1445    2906
       1446    2907
       1447    2908


In [22]:
numericals = ['LotFrontage', 'LotArea', 'OverallQual', 
             'OverallCond','YearBuilt', 'YearRemodAdd', 
             'MasVnrArea', 'ExterQual', 'ExterCond', 
             'BsmtQual', 'BsmtCond', 'BsmtFinSF1',
             'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 
             'HeatingQC', '1stFlrSF', '2ndFlrSF', 
             'LowQualFinSF','GrLivArea', 'BsmtFullBath', 
             'BsmtHalfBath', 'FullBath', 'HalfBath',
             'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
             'TotRmsAbvGrd','Fireplaces', 'FireplaceQu',
             'GarageYrBlt','GarageCars', 'GarageArea',
             'GarageQual', 'GarageCond','WoodDeckSF',
             'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
             'ScreenPorch', 'PoolArea', 'PoolQC', 
             'MiscVal','YrSold'] 

In [23]:
grades = ['OverallQual', 'OverallCond', 'ExterQual',
          'ExterCond', 'BsmtQual', 'BsmtCond',
          'HeatingQC', 'KitchenQual', 'FireplaceQu',
          'GarageQual', 'GarageCond', 'PoolQC']

In [24]:
literal = ['Ex', 'Gd', 'TA', 'Fa', 'Po']
num = [9, 7, 5, 3, 2]
data[grades] = data[grades].replace(dict(zip(literal, num)))

In [26]:
categoricals = data.drop(numericals, axis=1).columns
len(categoricals)

35

По возможности избавимся от значений NaN.

In [27]:
data.isnull().sum()[data.isnull().sum() > 0]

MSZoning           4
LotFrontage      486
Alley           2721
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        24
MasVnrArea        23
BsmtQual          81
BsmtCond          82
BsmtExposure      82
BsmtFinType1      79
BsmtFinSF1         1
BsmtFinType2      80
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
Electrical         1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu     1420
GarageType       157
GarageYrBlt      159
GarageFinish     159
GarageCars         1
GarageArea         1
GarageQual       159
GarageCond       159
PoolQC          2909
Fence           2348
MiscFeature     2814
SaleType           1
dtype: int64

В числовых переменных NaN можно интерпретировать как 0, в категориальных - как строку. 

In [28]:
cats = data.columns[data.dtypes == 'object']
nums = list(set(data.columns) - set(cats))

# Be sure the category 'None' is also handled here
data['MasVnrType'].replace({'None': np.nan}, inplace=True)

data[cats] = data[cats].fillna('0')
data[nums] = data[nums].fillna(0)

Меняем класс для двух переменных, чтобы потом создать dummies.

In [30]:
data['MSSubClass'] = data['MSSubClass'].astype('object', copy=False)
data['MoSold'] = data['MoSold'].astype('object', copy=False)

Уберем категориальные переменные с одним значением, остальные закодируем

In [35]:
price = np.log1p(train['SalePrice'])
categorical_data = pd.concat((data.loc['train'][categoricals], price), axis=1)

In [36]:
for feat in categorical_data.columns[:-1]:    
    uni = categorical_data.groupby(feat).mean().sort_values(by='SalePrice').index
    if (len(uni) < 2):
            data.drop(feat, axis=1, inplace=True)
    elif len(uni) < 3:
        print("{}: {}".format(feat, uni))
        data[feat].replace({uni[0]: 0, uni[1]: 1}, inplace=True)
        data[feat] = data[feat].astype('int8')
    else:
        data[feat] = data[feat].astype('category')

Street: Int64Index([0, 1], dtype='int64', name='Street')
Utilities: Int64Index([0, 1], dtype='int64', name='Utilities')
CentralAir: Int64Index([0, 1], dtype='int64', name='CentralAir')


In [40]:
alldata = pd.get_dummies(data)

In [45]:
X_test = alldata.loc['test']
X_train = alldata.loc['train']

y_train = price

Standardized scaling

In [46]:
m = X_train.mean()
std = X_train.std()

X_train = (X_train - m) / std
X_test = (X_test - m) / std

Lasso Regression, чтобы избавиться от обилия лишних коэффициентов.

In [49]:
Ls = LassoCV()

# Train the model using the training sets
Ls.fit(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [57]:
X_train.isnull().sum()[X_train.isnull().sum() > 0]

Series([], dtype: int64)

In [52]:
names = [
    'MSSubClass_150',    
    'MSZoning_0', 
    'Exterior1st_0',     
    'Exterior2nd_0',     
    'Functional_0',      
    'SaleType_0'      
]

In [60]:
X_train = X_train.drop(names, axis = 1)
X_test = X_test.drop(names, axis = 1)

KeyError: "['MSSubClass_150' 'MSZoning_0' 'Exterior1st_0' 'Exterior2nd_0'\n 'Functional_0' 'SaleType_0'] not found in axis"

In [64]:
Lasso_model = LassoCV()

Lasso_model.fit(X_train, y_train)

LassoCV(alphas=None, copy_X=True, cv=None, eps=0.001, fit_intercept=True,
    max_iter=1000, n_alphas=100, n_jobs=1, normalize=False, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
    verbose=False)

In [65]:
submit = pd.DataFrame({'Id': testID, 'SalePrice': np.exp(Lasso_model.predict(X_test))})
submit.to_csv('submission.csv', index=False)